# 추천시스템 + 콘텐츠 기반 필터링 실습
파이썬 완벽 머신러닝 가이드 참조

### 유형
1. **콘텐츠 기반 필터링**(Content based filtering)  
: 사용자가 특정 아이템을 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 아이템 추천
2. **협업 필터링**(Collaborative Filtering)
> 최근접 이웃(Nearest Neighbor) 협업 필터링  
> 잠재 요인(Latent Factor) 협업 필터링 : **행렬 분해(Matrix Factorization)** 이용

# SGD 기반 행렬분해(Matrix Factorization)

In [1]:
# 구현

import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P, Q 초기화, 잠재 요인 차원 = K

R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
             [np.NaN, 5, np.NaN, 3, 1], 
             [np.NaN, np.NaN, 3, 4, 4], 
             [5,2,1,2,np.NaN]]) # NaN값 포함해 행렬 생성

num_users, num_items = R.shape

k=3 # 잠재변수 차원은 3!

# P와 Q의 행렬 크기 지정, 정규 분포 가진 임의의 값 입력

np.random.seed(1) #정규분포 생성

P = np.random.normal(scale=1./k, size=(num_users, k))
Q = np.random.normal(scale=1./k, size=(num_items, k))


In [2]:
# 실제 R 행렬과 예측 행렬(R hat)의 오차를 구하는 get_rmse() 함수 구하기

from sklearn.metrics import mean_squared_error

# 함수 정의 : 위치 인덱스 추출 > 조합된 예측행렬 값의 RMSE 반환
def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    # 두 개의 분해된 행렬 P와 Q^T의 내적 = 예측행렬 R hat 생성
    full_pred_matrix = np.dot(P, Q.T) # dot : 내적
    
    # 실제 R 행렬에서 NULL 이 아닌 값의 위치 인덱스를 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    # NULL 이 아닌 인덱스 병합
    R_non_zeros= R[x_non_zero_ind, y_non_zero_ind] #실제 행렬
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    # 예측 행렬
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros) 
    #실제 행렬-예측 행렬의 MSE(NULL 값 포함 열은 제외!)
    
    rmse = np.sqrt(mse)
    
    return rmse
    

In [3]:
# SGD 기반 행렬 분해

# R에서 NULL 값 제외한 데이터의 행렬 인덱스 추출
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) 
             for j in range(num_items) if R[i,j]>0]

steps = 1000 #SGD를 반복해서 업데이트할 횟수
learning_rate = 0.01 # 학습률
r_lambda = 0.01 #L2 Regularization 계수

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        
        # 실제값-예측값 차이 오류 구함
        eij = r-np.dot(P[i,:], Q[j,:].T)
        
        # regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate * (eij*Q[j,:] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate * (eij*P[i,:] - r_lambda*Q[i,:])
        
    rmse = get_rmse(R, P, Q, non_zeros)

    # 50번 반복할 때마다 rmse 출력 (step%50 = 0 일때마다!)
    
    if (step%50) == 0 :
        print("### iteration step :", step, "rmse:", rmse)

### iteration step : 0 rmse: 3.238798211736208
### iteration step : 50 rmse: 0.48807825833664215
### iteration step : 100 rmse: 0.15597380323609472
### iteration step : 150 rmse: 0.07365850716866373
### iteration step : 200 rmse: 0.04199195918203749
### iteration step : 250 rmse: 0.02767186168350607
### iteration step : 300 rmse: 0.020781470320062945
### iteration step : 350 rmse: 0.017453784121568065
### iteration step : 400 rmse: 0.015833102364042246
### iteration step : 450 rmse: 0.015011390718278547
### iteration step : 500 rmse: 0.014566776792220995
### iteration step : 550 rmse: 0.014307430237811445
### iteration step : 600 rmse: 0.014144107792374473
### iteration step : 650 rmse: 0.014033240209514816
### iteration step : 700 rmse: 0.013952367720194226
### iteration step : 750 rmse: 0.013889326826564246
### iteration step : 800 rmse: 0.013837258409376631
### iteration step : 850 rmse: 0.013792169225998492
### iteration step : 900 rmse: 0.013751675092128376
### iteration step : 95

In [4]:
# 예측행렬 출력

pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))  # \n 는 띄어쓰기

예측 행렬:
 [[3.990e+00 7.990e-01 1.304e+00 2.005e+00 1.639e+00]
 [6.948e+00 4.980e+00 1.032e+00 2.980e+00 9.960e-01]
 [6.480e+00 3.000e-03 2.984e+00 3.981e+00 3.993e+00]
 [4.976e+00 2.003e+00 1.006e+00 2.005e+00 1.093e+00]]


NUll 값이 아닌 것은 거의 비슷하게 채워지고, Null값은 새로운 예측값이 채워짐

## 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터
특정 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천

### 1) 기본 전처리

In [5]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('movies.csv')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [6]:
movies.shape

(4803, 20)

In [7]:
# 주요 칼럼만 추출해 새로운 Data Frame 만들기

movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]
movies_df.head(2)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."


근데 지금 genres, keywords 에 element가 리스트 안에 들어있는 딕셔너리 형식으로 되어있음.     
가공해줘야 할 것 같아!

In [8]:
# pandas 설정 변경은 set_option!

pd.set_option('max_colwidth', 100) # 가로 너비를 100으로 넓게 설정! 

In [9]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [10]:
# name 다음에 오는 것들만 남겨야 함 (개별 장르를 파이썬 리스트 객체로 추출)
# literal_eval() 함수 : 문자열이 의미하는 list[dist1, dist2] 객체로 만들 수 있음. 

from ast import literal_eval # ast 모듈? 
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [11]:
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...",7.2,11800,150.437577,"[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."


In [12]:
# 장르명만 리스트 개체 추출 ('name'값에 해당하는 값을 추출)
# apply lambda 이용 : name 에 해당하는 값 반환! 

movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[y['name'] for y in x])

In [13]:
movies_df[['genres', 'keywords']][:1] # 이거 0이라고 쓰면 안된다? ;;

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


## 2) 장르 콘텐츠 유사도 측정
장르는 현재 리스트로 구성됨!   
> 1) genres 를 **문자열**로 변경한다.(현재 리스트임;;)   
> 2) 이를 **CountVectorizer** 로 **피쳐 벡터화**한다. (Count 기반 벡터화 변환)    
> 3) genres 문자열과 피쳐 벡터화 한 데이터 세트를 **코사인 유사도**로 변경한다.    
> 4) 유사도가 높은 영화 중 평점이 높은 순으로 영화를 추천한다.    

In [14]:
# 1) genres 를 문자열로 변경 > 새로운 열 지정 

movies_df['genres_literal'] = movies_df['genres'].apply(lambda x:(' ').join(x))
# join(x) : 문자열 결합
# 공백 문자로 word 단위가 구분되는 문자열로 변환 (CountVectorizer 적용)

In [15]:
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",Action Adventure Fantasy Science Fiction


In [16]:
# 2) CountVectorizer 로 피쳐화

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


4803개의 레코드와 276개의 개별 단어 피처로 구성된 피처 벡터 행렬 생성

In [17]:
# 3) cosine_similarity() 이용해 코사인 유사도 계산

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)

(4803, 4803)


In [18]:
print(genre_sim) # movies_df 의 행별 장르 유사도 값을 가지고 있음. 

[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


장르 기준으로 콘텐츠 기반 필터링을 수행하려면,      
movies_df의 개별 레코드에 대해 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야함.    
> 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 대상 행의 **위치 인덱스 값** 얻기 **( genre_sim.argsort()[:, ::-1] )** 이용해 위치 인덱스 값 가져오기

In [19]:
# argsort() 이용 : 작은 값부터 순서대로 데이터의 index를 반환해줌

genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] 
# ::-1 거꾸로 정렬 : 유사도 높은 순!

print(genre_sim_sorted_ind[:1]) # 0번 레코드의 비교 행 위치 인덱스만 추출

[[   0 3494  813 ... 3038 3037 2401]]


자신의 0번 레코드를 제외하면 3494, 813,,, 레코드 순으로 유사도가 높다!

### 3) 장르 콘텐츠 필터링을 통한 영화 추천
장르 유사도에 따라 영화를 추천하는 함수 생성

In [20]:
# find_sim_movie() 함수 생성

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # title_name 값인 DataFrame 추출
    title_movie = df[df['title']==title_name]
    
    # title_name 을 가진 index 객체를 ndarray로 반환
    title_index = title_movie.index.values # values 꼭 붙여주기
    
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로
    # top_n개의 index 추출
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index 출력. (이는 2차원 데이터임)
    print(similar_indexes)
    
    # but 데이터 프레임에서 index로 사용하기 위해 1차원 array로 변경 
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]
    

In [21]:
# 유사도 출력

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


vote_average는 평점인데, 평점이 0인 영화도 추천되고 좀 이상해보임. 개선 필요..!
> **평점정보 (vote_average)값** 이용  
> but 소수 관객이 평점 잘주면,, 공정한 평점이 아님.    
> 따라서 **sort_values()** 를 이용해 평점('vote_average') 오름차순으로 movies_df를 정렬해 10개만 출력해보겠다!

In [22]:
# 오름차순 : ascending = False
# sort_values() : 정렬

movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


무슨 듣보 영화인데 평점 준사람 한명밖에 없는 영화가 1위중임....
> **가중평균** 으로 해결해볼까??!?

In [25]:
movies_df.columns

Index(['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity',
       'keywords', 'overview', 'genres_literal'],
      dtype='object')

In [26]:
# 상위 60% 값 : quantile()로 추출 (사분위수)

C = movies_df['vote_average'].mean() # 영화 전체 평균 평점
m = movies_df['vote_count'].quantile(0.6) # 상위 60%

print('C:', round(C, 3), 'm:', round(m,3)) # round( ,n) : n째자리까지 반올림

C: 6.092 m: 370.2


In [35]:
# 기존 평점을 새로운 가중 평점으로 변경하는 함수 생성

percentile = 0.6
m = movies['vote_count'].quantile(percentile) # 투표횟수에 가중치 부여
C = movies['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m))*R) + ((m/(m+v))*C) # 가중평균 공식

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
# apply() 함수 인자로 입력한다!****


In [36]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


이제 새롭게 정의된 기준으로 영화 추출!!     
장르 유사성이 높은 영화를 2배수만큼 후보군으로 선정한 뒤,     
**weighted_vote 칼럼** 값이 높은 순으로 top_n만큼 추출하는 식으로 find_sim_movie()함수 변경

In [37]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n 2배에 해당하는 장르 유사성 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1) # reshape(-1) : 1차원 배열 반환
    
    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 가 높은 순으로 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [38]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
